# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 10 2022 8:30AM,243816,10,CTF0011358,"Citieffe, Inc.",Released
1,Aug 10 2022 8:30AM,243816,10,CTF0011359,"Citieffe, Inc.",Released
2,Aug 10 2022 8:30AM,243822,10,6822271,"Senseonics, Incorporated",Released
3,Aug 9 2022 4:10PM,243823,22,623661,"NBTY Global, Inc.",Released
4,Aug 9 2022 3:54PM,243821,21,50661177,"NBTY Global, Inc.",Released
5,Aug 9 2022 3:50PM,243818,102,PBR-080922-02,"Senseonics, Incorporated",Released
6,Aug 9 2022 3:48PM,243817,15,8381346,"Carwin Pharmaceutical Associates, LLC",Released
7,Aug 9 2022 3:20PM,243813,10,0085930698,ISDIN Corporation,Released
8,Aug 9 2022 3:20PM,243813,10,0085930704,ISDIN Corporation,Released
9,Aug 9 2022 3:20PM,243813,10,0085930718,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
34,243817,Released,1
35,243818,Released,1
36,243821,Released,1
37,243822,Released,1
38,243823,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
243817,NaN,NaN,1.0
243818,NaN,NaN,1.0
243821,NaN,NaN,1.0
243822,NaN,NaN,1.0
243823,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243722,0.0,0.0,1.0
243723,33.0,24.0,51.0
243732,0.0,0.0,1.0
243744,10.0,5.0,0.0
243755,0.0,0.0,8.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243722,0,0,1
243723,33,24,51
243732,0,0,1
243744,10,5,0
243755,0,0,8


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243722,0,0,1
1,243723,33,24,51
2,243732,0,0,1
3,243744,10,5,0
4,243755,0,0,8


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243722,,,1
1,243723,33,24,51
2,243732,,,1
3,243744,10,5,
4,243755,,,8


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 10 2022 8:30AM,243816,10,"Citieffe, Inc."
2,Aug 10 2022 8:30AM,243822,10,"Senseonics, Incorporated"
3,Aug 9 2022 4:10PM,243823,22,"NBTY Global, Inc."
4,Aug 9 2022 3:54PM,243821,21,"NBTY Global, Inc."
5,Aug 9 2022 3:50PM,243818,102,"Senseonics, Incorporated"
6,Aug 9 2022 3:48PM,243817,15,"Carwin Pharmaceutical Associates, LLC"
7,Aug 9 2022 3:20PM,243813,10,ISDIN Corporation
14,Aug 9 2022 3:17PM,243812,10,Yusen – 3D Matrix
15,Aug 9 2022 3:15PM,243811,10,"Nextsource Biotechnology, LLC"
18,Aug 9 2022 3:06PM,243808,20,"ACI Healthcare USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 10 2022 8:30AM,243816,10,"Citieffe, Inc.",,,2
1,Aug 10 2022 8:30AM,243822,10,"Senseonics, Incorporated",,,1
2,Aug 9 2022 4:10PM,243823,22,"NBTY Global, Inc.",,,1
3,Aug 9 2022 3:54PM,243821,21,"NBTY Global, Inc.",,,1
4,Aug 9 2022 3:50PM,243818,102,"Senseonics, Incorporated",,,1
5,Aug 9 2022 3:48PM,243817,15,"Carwin Pharmaceutical Associates, LLC",,,1
6,Aug 9 2022 3:20PM,243813,10,ISDIN Corporation,,,7
7,Aug 9 2022 3:17PM,243812,10,Yusen – 3D Matrix,,,1
8,Aug 9 2022 3:15PM,243811,10,"Nextsource Biotechnology, LLC",,,3
9,Aug 9 2022 3:06PM,243808,20,"ACI Healthcare USA, Inc.",,,55


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 10 2022 8:30AM,243816,10,"Citieffe, Inc.",2,,
1,Aug 10 2022 8:30AM,243822,10,"Senseonics, Incorporated",1,,
2,Aug 9 2022 4:10PM,243823,22,"NBTY Global, Inc.",1,,
3,Aug 9 2022 3:54PM,243821,21,"NBTY Global, Inc.",1,,
4,Aug 9 2022 3:50PM,243818,102,"Senseonics, Incorporated",1,,
5,Aug 9 2022 3:48PM,243817,15,"Carwin Pharmaceutical Associates, LLC",1,,
6,Aug 9 2022 3:20PM,243813,10,ISDIN Corporation,7,,
7,Aug 9 2022 3:17PM,243812,10,Yusen – 3D Matrix,1,,
8,Aug 9 2022 3:15PM,243811,10,"Nextsource Biotechnology, LLC",3,,
9,Aug 9 2022 3:06PM,243808,20,"ACI Healthcare USA, Inc.",55,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 10 2022 8:30AM,243816,10,"Citieffe, Inc.",2,,
1,Aug 10 2022 8:30AM,243822,10,"Senseonics, Incorporated",1,,
2,Aug 9 2022 4:10PM,243823,22,"NBTY Global, Inc.",1,,
3,Aug 9 2022 3:54PM,243821,21,"NBTY Global, Inc.",1,,
4,Aug 9 2022 3:50PM,243818,102,"Senseonics, Incorporated",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 10 2022 8:30AM,243816,10,"Citieffe, Inc.",2.0,NaN,NaN
1,Aug 10 2022 8:30AM,243822,10,"Senseonics, Incorporated",1.0,NaN,NaN
2,Aug 9 2022 4:10PM,243823,22,"NBTY Global, Inc.",1.0,NaN,NaN
3,Aug 9 2022 3:54PM,243821,21,"NBTY Global, Inc.",1.0,NaN,NaN
4,Aug 9 2022 3:50PM,243818,102,"Senseonics, Incorporated",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 10 2022 8:30AM,243816,10,"Citieffe, Inc.",2.0,0.0,0.0
1,Aug 10 2022 8:30AM,243822,10,"Senseonics, Incorporated",1.0,0.0,0.0
2,Aug 9 2022 4:10PM,243823,22,"NBTY Global, Inc.",1.0,0.0,0.0
3,Aug 9 2022 3:54PM,243821,21,"NBTY Global, Inc.",1.0,0.0,0.0
4,Aug 9 2022 3:50PM,243818,102,"Senseonics, Incorporated",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2437976,39.0,6.0,0.0
102,243818,1.0,0.0,0.0
12,243783,0.0,1.0,0.0
15,731295,60.0,24.0,33.0
19,487563,14.0,18.0,13.0
20,1706470,126.0,8.0,12.0
21,487543,2.0,0.0,0.0
22,731429,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2437976,39.0,6.0,0.0
1,102,243818,1.0,0.0,0.0
2,12,243783,0.0,1.0,0.0
3,15,731295,60.0,24.0,33.0
4,19,487563,14.0,18.0,13.0
5,20,1706470,126.0,8.0,12.0
6,21,487543,2.0,0.0,0.0
7,22,731429,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,39.0,6.0,0.0
1,102,1.0,0.0,0.0
2,12,0.0,1.0,0.0
3,15,60.0,24.0,33.0
4,19,14.0,18.0,13.0
5,20,126.0,8.0,12.0
6,21,2.0,0.0,0.0
7,22,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,39.0
1,102,Released,1.0
2,12,Released,0.0
3,15,Released,60.0
4,19,Released,14.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,19,20,21,22
Status,,,,,,,,
Completed,0.0,0.0,0.0,33.0,13.0,12.0,0.0,0.0
Executing,6.0,0.0,1.0,24.0,18.0,8.0,0.0,0.0
Released,39.0,1.0,0.0,60.0,14.0,126.0,2.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,19,20,21,22
0,Completed,0.0,0.0,0.0,33.0,13.0,12.0,0.0,0.0
1,Executing,6.0,0.0,1.0,24.0,18.0,8.0,0.0,0.0
2,Released,39.0,1.0,0.0,60.0,14.0,126.0,2.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,19,20,21,22
0,Completed,0.0,0.0,0.0,33.0,13.0,12.0,0.0,0.0
1,Executing,6.0,0.0,1.0,24.0,18.0,8.0,0.0,0.0
2,Released,39.0,1.0,0.0,60.0,14.0,126.0,2.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()